In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from pathlib import Path

In `config`:

    config = Config(
        max_length = [256, 256]  # in tokens, list for title vs abstract
        from_pretrained = "facebook/bart-base"
        model_type = "Twin"
    )

In [4]:
from smtag.config import config
assert config.max_length==[256, 256]
assert config.from_pretrained=='facebook/bart-base'
assert config.model_type=='Twin'

2022-08-12 06:51:11.835896: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-12 06:51:11.835950: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-12 06:51:13.914962: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-12 06:51:13.915133: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-12 06:51:13.915157: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-12 06:51:13.915186: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [5]:
config

Config(max_length=[256, 256], truncation=True, min_char_length=80, celery_batch_size=1000, from_pretrained='facebook/bart-base', model_type='Twin', nlp=<spacy.lang.en.English object at 0x7f712f288410>, fast=True, asynchr=True, twin_delimiter='###tt9HHSlkWoUM###')

In [6]:
from transformers import __version__
__version__

'4.15.0'

## Extracting examples for LM

In [7]:
from smtag.extract import ExtractorXML

In [8]:
# ! rm -fr /data/text/oapmc_twin
! rm -fr /data/text/emboj_twin
# ! rm -fr /data/text/mini_twin

In [9]:
corpus = "/data/xml/emboj_all"
text_examples = "/data/text/emboj_twin"
xpath = [".//article-meta/title-group/article-title", ".//abstract"]
sentence_level = False
keep_xml = False
inclusion_probability = 1.0

In [10]:
extractor_lm = ExtractorXML(
    corpus,
    destination_dir=text_examples,
    sentence_level=sentence_level,
    xpath=xpath,
    keep_xml=keep_xml,
    inclusion_probability=inclusion_probability
)

/data/text/emboj_twin created


In [11]:
extractor_lm.extract_from_corpus()

100%|██████████| 4/4 [00:24<00:00,  6.24s/it]


{PosixPath('/data/text/emboj_twin/train.txt'): 7991,
 PosixPath('/data/text/emboj_twin/eval.txt'): 2651,
 PosixPath('/data/text/emboj_twin/test.txt'): 2659}

same via CLI:


```bash
python -m smtag.cli.prepro.extract /data/xml/emboj_all /data/text/emboj_twin --xpath ".//article-meta/title-group/article-title" ".//abstract"
```

```bash
python -m smtag.cli.prepro.extract /data/xml/oapmc_articles /data/text/oapmc_twin --xpath ".//article-meta/title-group/article-title" ".//abstract"
```

```bash
python -m smtag.cli.prepro.extract /data/xml/emboj_all /data/text/emboj_trivial --xpath ".//article-meta/title-group/article-title" ".//article-meta/title-group/article-title"
```

## Preparing tokenized dataset for LM

In [12]:
from smtag.dataprep import PreparatorLM

In [13]:
tokenized_examples = "/data/json/emboj_twin"

In [14]:
# ! rm -fr /data/json/oapmc_twin
! rm -fr /data/json/emboj_twin

In [15]:
prep_lm = PreparatorLM(
    text_examples,
    tokenized_examples,
    max_length=config.max_length
)

/data/json/emboj_twin created


In [16]:
prep_lm.run()

Preparing: train


100%|██████████| 7991/7991 [01:49<00:00, 72.94it/s]


Preparing: eval


100%|██████████| 2651/2651 [00:30<00:00, 86.34it/s]


Preparing: test


100%|██████████| 2659/2659 [00:31<00:00, 85.23it/s]


same vie CLI:
    
```bash
python -m smtag.cli.lm.dataprep /data/text/oapmc_twin /data/json/oapmc_twin
```

```bash
python -m smtag.cli.lm.dataprep /data/text/emboj_twin /data/json/emboj_twin
```

```bash
python -m smtag.cli.lm.dataprep /data/text/emboj_trivial /data/json/emboj_trivial
```

## Train LM

In [ ]:
%autoreload 2

In [23]:
from smtag.train.train_lm import (
    train as train_lm,
    TrainingArgumentsLM
)

In [24]:
no_cache = True  # ALWAYS CHECK THIS IN CASE OF PROBLEMS!!!
loader_path = "./smtag/loader/loader_twin.py"
data_config_name = "NOLM"  # "SEQ2SEQ" to have also a language model
tokenizer = config.tokenizer  # tokenizer has to be the same application-wide
model_type = "Twin"
from_pretrained = config.from_pretrained

In [25]:
tokenizer.mask_token = '<mask>'  # why is this here? maybe because in case of character-level tokenizer
# tokenizer.unk_token = '<unk>'

In [26]:
training_args_tokcl = TrainingArgumentsLM(
    num_train_epochs = 1,
    logging_steps = 10,  # 100 or 1000 for large oapmc_twin
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
)
training_args_tokcl

TrainingArgumentsLM(output_dir='/lm_models', overwrite_output_dir=True, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=64, per_device_eval_batch_size=64, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/lm_models/runs/Aug12_07-30-45_0051b612e94f', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=10, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=5, save_on_each_node=False, no_cuda=False, seed=42, bf16

In [27]:
train_lm(
    training_args_tokcl,
    loader_path,
    data_config_name,
    tokenized_examples,
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

tokenizer vocab size: 50265

Loading datasets found in /data/json/emboj_twin.
using ./smtag/loader/loader_twin.py as dataset loader.


  0%|          | 0/3 [00:00<?, ?it/s]


Training with 7991 examples.
Evaluating on 2651 examples.


AttributeError: 'Twin' object has no attribute 'models'

#### With CLI:

```bash
python -m smtag.cli.lm.train smtag/loader/loader_twin.py NOLM --data_dir /data/json/oapmc_twin --per_device_train_batch_size=64 --per_device_eval_batch_size=64 --logging_steps=100 --num_train_epochs=100 --save_steps=12000
```

```bash
python -m smtag.cli.lm.train smtag/loader/loader_twin.py SEQ2SEQ --data_dir /data/json/emboj_twin --per_device_train_batch_size=16 --per_device_eval_batch_size=16 --logging_steps=100 --num_train_epochs=1 --no_cache 
```